In [1]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer

c:\Users\izzfa\anaconda3\envs\DC3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Read the data and perform preprocessing

df = pd.read_csv("data\df_all_cleaned.csv") # Read data into 'df' dataframe
print(df.shape) # Print dataframe shape

summary = df["post_extrovert"].tolist() # Create a list containing all article summaries

df

In [ ]:
#Run this to get the zero shot classification result. Will take awhile.
results = []

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33", use_fast=False)
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33")

for text in tqdm(summary):

    context = text
    sequence_to_classify = context

     # Tokenize the text and truncate if needed
    inputs = tokenizer(sequence_to_classify, return_tensors="pt", truncation=True, max_length=512)

    # Perform zero-shot classification
    output = classifier([sequence_to_classify], candidate_labels=["Introvert", 'Extrovert'], multi_label=False)
    # candidate_labels = ["Introvert", 'Extrovert']
    # output = classifier(sequence_to_classify, candidate_labels, multi_label=False)

    results.append(output)

In [ ]:
# Sorting the labels and scores for DeBERTa

labels = []
scores =[]

for res in results:
    labels.append(res['labels'][0])
    scores.append(res['scores'][0])

df['Zeroshot'] = labels
df['Scores for Zeroshot'] = scores